<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="chevalblanc/gpt-4o-mini"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


In [2]:
!pip install openai --quiet

In [3]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [4]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [5]:
from pydantic import Field, BaseModel

class Response(BaseModel):
   action: str = Field(description="the agent to select or \"respond_to_user\" when there are no agents to select from or there is no next_action")
   input: str = Field(description="the input for this agent")
   next_action: str = Field(description="next action after this action step")


In [7]:
def call(system_prompt: str, message: str, model: str = modelid ) -> str:
  completion = llm.chat.completions.parse(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
      response_format=Response
  )

  return completion.choices[0].message.parsed

In [9]:
def orchestrator(userinput: str, memory: list[str] = []) -> Response:


  prompt = """
                Use the context from memory to plan next steps.
                Context:
                {context}

                You need will use the context provided and the user's input to classify the intent select the appropriate agent.
                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions and input schema:

               current_time_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               weather_agent - Provides the current weather for a given city.

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the action "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with action "respond_to_user" only when there are no agents to select from or there is no next_action


  """

  for i in range (0,5):

    step = call ("You are an expert intent classifier.", prompt.format(userinput=userinput,context="\n".join(memory)))
    print (step)

    if step.action == "current_time_agent":
      memory.append("Response from agent was: the current time is 10:00 am")
    if step.action == "schedule_agent":
      return orchestrator(step.input)
    if step.action == "weather_agent":
      memory.append("Response from agent was: The weather in New York is sunny width 39 degress")
    if step.action == "respond_to_user":
      return step.input


In [10]:
result = orchestrator ("What is the current time and weather in New York ?")
result

agent_name='current_time_agent' input='<start of conversation>' next_action='send_agent_input'


AttributeError: 'Response' object has no attribute 'action'

In [ ]:
result = orchestrator ("Please schedule a date in my calendar starting in 1 hour.")
result












A Draft for a first version that used text instead of structured outputs





In [ ]:
def orchestrator_text_based(userinput: str) -> str:

  response_format = {"action":"", "input":"", "next_action":""}

  prompt = """
                Use the context from memory to plan next steps.
                Context:
                []

                You are an expert intent classifier.
                You need will use the context provided and the user's input to classify the intent select the appropriate agent.
                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions:

               time_tool_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               calendar_agent - Checks your calendar or schedules a date in your calendar at a given time

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the action "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with "respond_to_user" only when there are no agents to select from or there is no next_action
                - You will return the agent name in the form of {response_format}
                - Always return valid JSON like {response_format} and nothing else.
  """
  return call ("You are an expert intent classifier.", prompt.format(userinput=userinput, response_format=response_format))